<a href="https://colab.research.google.com/github/mantle-bearer/FAW-Detection-Capstone/blob/main/LAS_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
mkdir faw_capstone/

SyntaxError: invalid syntax (ipython-input-901507306.py, line 1)

In [11]:
cd faw_capstone

/content/faw_capstone


In [12]:
mkdir notebooks src data models artifacts

In [15]:
!echo -e "torch\ntorchvision\nonnxruntime\nonnx\nscikit-learn\nmatplotlib\npillow\ntimm\nultralytics\nflask\npython-telegram-bot\nroboflow" > requirements.txt

In [16]:
!pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cu118
!pip install -q timm onnx onnxruntime scikit-learn matplotlib pillow tqdm roboflow
# Optional for detection:
!pip install -q ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.6 MB/s eta 0:00:00


In [18]:
# Colab cell 2 - mount Drive to save model artifacts permanently
from google.colab import drive
drive.mount('/content/drive')
ARTIFACT_DIR = "/content/drive/MyDrive/FAW_capstone"
!mkdir -p "{ARTIFACT_DIR}"
print("Artifacts will be saved to:", ARTIFACT_DIR)


Mounted at /content/drive
Artifacts will be saved to: /content/drive/MyDrive/FAW_capstone


In [1]:
# Colab cell 3 - download dataset zip and unzip
DATA_ZIP_URL = "https://drive.google.com/drive/folders/1g2PxrzbgyrTwA5LIdg8Y7MuYux-hQwvg?usp=sharing"  # <-- REPLACE
!wget -q -O /content/dataset.zip "{DATA_ZIP_URL}"
!unzip -q /content/dataset.zip -d /content/dataset
!find /content/dataset -maxdepth 3 -type d -print


[/content/dataset.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/dataset.zip or
        /content/dataset.zip.zip, and cannot find /content/dataset.zip.ZIP, period.
find: ‘/content/dataset’: No such file or directory


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
import os

# Path to your dataset in Drive
DATASET_DIR = "/fall armyworm identification/fall armyworm identification/"  # Change this to your actual folder name

# Make sure the folder exists
if os.path.exists(DATASET_DIR):
    print("✅ Dataset folder found:", DATASET_DIR)
else:
    print("⚠️ Dataset folder not found. Please check the path.")


⚠️ Dataset folder not found. Please check the path.


In [6]:
import os

for root, dirs, files in os.walk(DATASET_DIR):
    level = root.replace(DATASET_DIR, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for f in files[:5]:  # show first 5 files per folder
        print(f"{subindent}{f}")


In [7]:
!pip install torch torchvision torchaudio --quiet

In [8]:
!pip install ultralytics --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.0 MB/s eta 0:00:00


In [10]:
import os, shutil, random
from glob import glob

source_dir = DATASET_DIR
train_dir = "/content/dataset/train"
val_dir = "/content/dataset/val"

if not os.path.exists(train_dir):
    os.makedirs(train_dir)
    os.makedirs(val_dir)

    all_images = glob(os.path.join(source_dir, "*.jpg")) + glob(os.path.join(source_dir, "*.png"))
    random.shuffle(all_images)

    split = int(0.8 * len(all_images))
    train_images = all_images[:split]
    val_images = all_images[split:]

    for img in train_images:
        shutil.copy(img, train_dir)
    for img in val_images:
        shutil.copy(img, val_dir)

    print("✅ Dataset split into train and val")
else:
    print("✅ Already prepared")


✅ Already prepared


In [12]:
from ultralytics import YOLO

# Load YOLOv8 model (you can also use 'yolov8n.pt', 'yolov8s.pt', etc.)
model = YOLO('yolov8n.pt')

# Train on your dataset
model.train(
    data=f"{DATASET_DIR}/data.yaml",  # Path to dataset config file
    epochs=50,
    imgsz=640,
    batch=8
)


Ultralytics 8.3.223 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/fall armyworm identification/fall armyworm identification/test/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, pa

RuntimeError: Dataset '/fall armyworm identification/fall armyworm identification/test/data.yaml' error ❌ '/fall armyworm identification/fall armyworm identification/test/data.yaml' does not exist